In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import re

In [22]:
# Utils
start_date = '2024-06-20 00:00:00'
end_date = '2024-07-20 23:59:59'

def inRange(start_date, end_date, time):
    return start_date <= time <= end_date

before_start=True

In [23]:
test = '2024-06-19 23:59:59'
inRange(start_date, end_date, test)

False

In [29]:
# 게시물 검색 페이지 (page_num으로 지정 가능)
driver = webdriver.Chrome() 
page_num=0
driver.get(f'https://www.clien.net/service/search?q=%EB%A5%B4%EB%85%B8&sort=recency&p={page_num}&boardCd=&isBoard=false')
time.sleep(1)

- 검색했을 때, 최대 50페이지만 보여줌

In [11]:
# get post id
# `data-board-sn` 값을 가진 요소 찾기

In [30]:
# per page
news_titles_elem = driver.find_elements(By.CLASS_NAME, "subject_fixed")
created_ats_elem = driver.find_elements(By.CLASS_NAME, "timestamp")
authors_elem = driver.find_elements(By.CLASS_NAME, "nickname")

titles = []
hrefs = []
created_ats = []
authors = []

# get title
for i in news_titles_elem:
    titles.append(i.text)
    hrefs.append(i.get_attribute('href'))

# get datetime
for i in created_ats_elem:
    created_ats.append(i.get_attribute('textContent'))

# get author name
images_elem = driver.find_elements(By.CSS_SELECTOR, 'span.nickname img') # for gif nickname
gif_nick_cnt = 0
for idx, elem in enumerate(authors_elem):
    nickname = elem.text
    if nickname == '':
        nickname = images_elem[gif_nick_cnt].get_attribute('alt')
        gif_nick_cnt+=1
    authors.append(nickname)

In [32]:
# per contents
for href in hrefs:
    # open new page
    driver.get(href)
    time.sleep(3)

    # get post_id
    meta_tag = driver.find_element(By.CSS_SELECTOR, 'meta[property="url"]')    
    url_value = meta_tag.get_attribute('content')
    match = re.search(r'/(\d+)$', url_value)
    post_id = match.group(1)

    # get likes
    likes_element = driver.find_element(By.CSS_SELECTOR, 'a.symph_count > strong')
    likes = likes_element.text

    # get view count
    view_count_elem = driver.find_element(By.CSS_SELECTOR, 'span.view_count > strong')
    view_count = int(view_count_elem.text.replace(',', ''))

    # get content
    class_name = 'post_article' 
    content_elems = driver.find_elements(By.CSS_SELECTOR, f'.{class_name} p, .{class_name} h1, \
    .{class_name} h2, .{class_name} h3, .{class_name} h4, .{class_name} h5, .{class_name} h6')
    content=""
    for content_elem in content_elems:
        content += content_elem.text
    
    # get comments
    cmt_authors = []
    cmt_contents = []
    cmt_post_ids = []    
    
    cmt_ad_banner_elem = driver.find_element(By.CLASS_NAME, "comment.ad_banner")
    cmt_authors_elem = cmt_ad_banner_elem.find_elements(By.CLASS_NAME, "nickname")
    cmt_gif_authors_elem = cmt_ad_banner_elem.find_elements(By.CSS_SELECTOR, 'span.nickname img') # for gif nickname
    cmt_contents_elem = cmt_ad_banner_elem.find_elements(By.CLASS_NAME, "comment_view")
    cmt_gif_authors_cnt = 0
    
    assert len(cmt_authors_elem)==len(cmt_contents_elem)    
    for idx in range(len(cmt_authors_elem)):
        cmt_author = cmt_authors_elem[idx].text
        if cmt_author=='': # for gif nickname
            cmt_author = cmt_gif_authors_elem[cmt_gif_authors_cnt].get_attribute('alt')
            cmt_gif_authors_cnt += 1
        cmt_authors.append(cmt_author)        
        cmt_contents.append(cmt_contents_elem[idx].text)
        cmt_post_ids.append(post_id)

    # for debugging
    # print("content: ", content)
    # print("post_id: ", post_id)
    # print("cmt_authors: ", cmt_authors)
    # print("cmt_contents: ", cmt_contents)
    # print("cmt_post_ids: ", cmt_post_ids)
    # print("\n\n")

0
3487
5
7715
8
16339
0
4552


KeyboardInterrupt: 

In [ ]:
driver.quit()